<a href="https://colab.research.google.com/github/ganiosh92/DA-ecommerce-clickstream-analytics/blob/main/clickstream_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pwd
%cd /content/drive/My Drive/Sample-Data-From-Web/ecommerce_customer_behavior

/content
/content/drive/My Drive/Sample-Data-From-Web/ecommerce_customer_behavior


In [3]:
import pandas as pd
import numpy as np
import json
from pandas import json_normalize

In [4]:
customer = pd.read_csv('customer.csv')
clickstream = pd.read_csv('click_stream.csv')
transactions = pd.read_csv('transactions.csv')
product = pd.read_csv('product.csv', on_bad_lines='skip')

In [5]:
clickstream.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12833602 entries, 0 to 12833601
Data columns (total 6 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   session_id      object
 1   event_name      object
 2   event_time      object
 3   event_id        object
 4   traffic_source  object
 5   event_metadata  object
dtypes: object(6)
memory usage: 587.5+ MB


In [6]:
clickstream.head()

,session_id,event_name,event_time,event_id,traffic_source,event_metadata
0,fb0abf9e-fd1a-44dd-b5c0-2834d5a4b81c,HOMEPAGE,2019-09-06T15:54:32.821085Z,9c4388c4-c95b-4678-b5ca-e9cbc0734109,MOBILE,NaN
1,fb0abf9e-fd1a-44dd-b5c0-2834d5a4b81c,SCROLL,2019-09-06T16:03:57.821085Z,4690e1f5-3f99-42d3-84a5-22c4c4d8500a,MOBILE,NaN
2,7d440441-e67a-4d36-b324-80ffd636d166,HOMEPAGE,2019-09-01T12:05:10.322763Z,88aeaeb5-ec98-4859-852c-8abb483faf31,MOBILE,NaN
3,7d440441-e67a-4d36-b324-80ffd636d166,ADD_TO_CART,2019-09-01T12:06:33.322763Z,934e306e-ecc6-472f-9ccb-12c8536910a2,MOBILE,"{'product_id': 15315, 'quantity': 4, 'item_pri..."
4,7d440441-e67a-4d36-b324-80ffd636d166,BOOKING,2019-09-01T12:15:29.425431Z,9f4767a1-40fa-4c9c-9524-dfad18634d56,MOBILE,{'payment_status': 'Success'}


In [7]:
#create a function to convert event_metadata column to JSON format
def func(a):
  return(json.loads(a.replace("'",'"')))

In [8]:
#flatten the event_metadata column

#drop rows with na values
clickstream_no_na = clickstream.dropna().copy()

#create a new column and fill it with dummy value = 1
clickstream_no_na['test_col'] = "1"

#apply the function to convert event_metadata column values from string to JSON
clickstream_no_na['test_col'] = clickstream_no_na['event_metadata'].apply(func)

#impute the flattened columns to the old dataset
clickstream_no_na_flattened = pd.concat([clickstream_no_na, json_normalize(clickstream_no_na['test_col'])], axis=1)

In [9]:
clickstream_no_na_flattened.head()

,session_id,event_name,event_time,event_id,traffic_source,event_metadata,test_col,product_id,quantity,item_price,payment_status,search_keywords,promo_code,promo_amount
3,7d440441-e67a-4d36-b324-80ffd636d166,ADD_TO_CART,2019-09-01T12:06:33.322763Z,934e306e-ecc6-472f-9ccb-12c8536910a2,MOBILE,"{'product_id': 15315, 'quantity': 4, 'item_pri...","{'product_id': 15315, 'quantity': 4, 'item_pri...",6133.0,2.0,249443.0,NaN,NaN,NaN,NaN
4,7d440441-e67a-4d36-b324-80ffd636d166,BOOKING,2019-09-01T12:15:29.425431Z,9f4767a1-40fa-4c9c-9524-dfad18634d56,MOBILE,{'payment_status': 'Success'},{'payment_status': 'Success'},6721.0,1.0,134504.0,NaN,NaN,NaN,NaN
5,7d440441-e67a-4d36-b324-80ffd636d166,SEARCH,2019-09-01T12:07:00.322763Z,c952142b-4fe9-4694-ad7f-21a5d1bed9ca,MOBILE,{'search_keywords': 'Dress Kondangan'},{'search_keywords': 'Dress Kondangan'},57622.0,1.0,384716.0,NaN,NaN,NaN,NaN
12,f7b5cd60-4f87-4b3c-8bde-c7da8930eec2,ADD_TO_CART,2019-09-09T05:06:03.425431Z,c1b42157-39a7-4521-a4dc-ed0200de301a,MOBILE,"{'product_id': 6133, 'quantity': 2, 'item_pric...","{'product_id': 6133, 'quantity': 2, 'item_pric...",NaN,NaN,NaN,Success,NaN,NaN,NaN
13,f7b5cd60-4f87-4b3c-8bde-c7da8930eec2,ADD_TO_CART,2019-09-09T05:23:36.425431Z,61cbe834-c585-406e-8eac-30f671e51fbb,MOBILE,"{'product_id': 6721, 'quantity': 1, 'item_pric...","{'product_id': 6721, 'quantity': 1, 'item_pric...",NaN,NaN,NaN,NaN,Tas Wanita,NaN,NaN


## TESTING

In [10]:
#Test
test = clickstream_no_na_flattened.copy()

In [11]:
#Test
test

,session_id,event_name,event_time,event_id,traffic_source,event_metadata,test_col,product_id,quantity,item_price,payment_status,search_keywords,promo_code,promo_amount
3,7d440441-e67a-4d36-b324-80ffd636d166,ADD_TO_CART,2019-09-01T12:06:33.322763Z,934e306e-ecc6-472f-9ccb-12c8536910a2,MOBILE,"{'product_id': 15315, 'quantity': 4, 'item_pri...","{'product_id': 15315, 'quantity': 4, 'item_pri...",6133.0,2.0,249443.0,NaN,NaN,NaN,NaN
4,7d440441-e67a-4d36-b324-80ffd636d166,BOOKING,2019-09-01T12:15:29.425431Z,9f4767a1-40fa-4c9c-9524-dfad18634d56,MOBILE,{'payment_status': 'Success'},{'payment_status': 'Success'},6721.0,1.0,134504.0,NaN,NaN,NaN,NaN
5,7d440441-e67a-4d36-b324-80ffd636d166,SEARCH,2019-09-01T12:07:00.322763Z,c952142b-4fe9-4694-ad7f-21a5d1bed9ca,MOBILE,{'search_keywords': 'Dress Kondangan'},{'search_keywords': 'Dress Kondangan'},57622.0,1.0,384716.0,NaN,NaN,NaN,NaN
12,f7b5cd60-4f87-4b3c-8bde-c7da8930eec2,ADD_TO_CART,2019-09-09T05:06:03.425431Z,c1b42157-39a7-4521-a4dc-ed0200de301a,MOBILE,"{'product_id': 6133, 'quantity': 2, 'item_pric...","{'product_id': 6133, 'quantity': 2, 'item_pric...",NaN,NaN,NaN,Success,NaN,NaN,NaN
13,f7b5cd60-4f87-4b3c-8bde-c7da8930eec2,ADD_TO_CART,2019-09-09T05:23:36.425431Z,61cbe834-c585-406e-8eac-30f671e51fbb,MOBILE,"{'product_id': 6721, 'quantity': 1, 'item_pric...","{'product_id': 6721, 'quantity': 1, 'item_pric...",NaN,NaN,NaN,NaN,Tas Wanita,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4289535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Success,NaN,NaN,NaN
4289536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bekas,NaN,NaN
4289537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19820.0,1.0,88918.0,NaN,NaN,NaN,NaN
4289538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Success,NaN,NaN,NaN
